# Training feature classifier 
### GTDB ref-seqs and taxonomy were obtained from Qiime2 website and produced by rescript

In [2]:
!conda install -y -c conda-forge -c bioconda -c qiime2 \
    -c https://packages.qiime2.org/qiime2/2023.5/tested/ -c defaults \
    xmltodict 'q2-types-genomics>2023.2' ncbi-datasets-pylib

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /home/meco/anaconda3/envs/qiime2-2023.5

  added / updated specs:
    - ncbi-datasets-pylib
    - q2-types-genomics[version='>2023.2']
    - xmltodict


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    argh-0.27.2                |     pyhd8ed1ab_0          42 KB  conda-forge
    biopython-1.81             |   py38h1de0b5d_0         2.6 MB  conda-fo

In [9]:
!conda install -y -c conda-forge -c bioconda git

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /home/meco/anaconda3/envs/qiime2-2023.5

  added / updated specs:
    - git


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    git-2.41.0                 | pl5321h86e50cf_0         9.4 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         9.4 MB

The following NEW packages will be INSTALLED:

  git                conda-forge/linux-64::git-2.41.0-pl5321h86e50cf_0 



                                                                                
Preparin

In [10]:
!pip install git+https://github.com/bokulich-lab/RESCRIPt.git

  Cloning https://github.com/bokulich-lab/RESCRIPt.git to /tmp/pip-req-build-cxiapq30
  Running command git clone --filter=blob:none --quiet https://github.com/bokulich-lab/RESCRIPt.git /tmp/pip-req-build-cxiapq30
  Resolved https://github.com/bokulich-lab/RESCRIPt.git to commit c7c6b4e8b0d7e2f1654806e733de5326757b442f
  Preparing metadata (setup.py) ... done
  Created wheel for rescript: filename=rescript-2023.5.0-py3-none-any.whl size=6318847 sha256=29d4356712a6edb248d0dda0052875f6569f218bfa6015a3d7d7bf169109865b
  Stored in directory: /tmp/pip-ephem-wheel-cache-j8ewrai7/wheels/a7/2f/ca/a4cfe2ac81c54ea686727a464c1029266233e0df67566e3523
Successfully built rescript


In [14]:
!qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [21]:
DOMS = {
    'Archaea': ['CAGYGCASCAGKCGMGAA', 'GGACTACVSGGGTATCTAAT'],
    'Bacteria': ['CAAGRGTTHGATYMTGGCTCAG', 'TGCTGCCTCCCGTAGGAGT']}

outdir = 'Classifier_GTDB'

In [18]:
!mkdir -p $outdir

for dom in DOMS:
    
    !qiime rescript get-gtdb-data \
        --p-version '214' \
        --p-domain $dom \
        --o-gtdb-taxonomy $outdir/$dom-gtdb-214-tax.qza \
        --o-gtdb-sequences $outdir/$dom-gtdb-214-seqs.qza

Saved FeatureData[Taxonomy] to: Classifier_GTDB/Archaea-gtdb-214.1-tax.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Archaea-gtdb-214.1-seqs.qza
Saved FeatureData[Taxonomy] to: Classifier_GTDB/Bacteria-gtdb-214.1-tax.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Bacteria-gtdb-214.1-seqs.qza


In [19]:
for dom in DOMS:
    
    !qiime rescript cull-seqs \
        --i-sequences $outdir/$dom-gtdb-214-seqs.qza \
        --o-clean-sequences $outdir/$dom-gtdb-214-seqs.qza

Saved FeatureData[Sequence] to: Classifier_GTDB/Archaea-gtdb-214.1-seqs.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Bacteria-gtdb-214.1-seqs.qza


In [20]:
for dom in DOMS:

    !qiime rescript dereplicate \
        --i-sequences $outdir/$dom-gtdb-214-seqs.qza  \
        --i-taxa $outdir/$dom-gtdb-214-tax.qza \
        --p-mode 'uniq' \
        --o-dereplicated-sequences $outdir/$dom-gtdb-214-seqs.qza \
        --o-dereplicated-taxa $outdir/$dom-gtdb-214-tax.qza

Saved FeatureData[Sequence] to: Classifier_GTDB/Archaea-gtdb-214-seqs.qza
Saved FeatureData[Taxonomy] to: Classifier_GTDB/Archaea-gtdb-214-tax.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Bacteria-gtdb-214-seqs.qza
Saved FeatureData[Taxonomy] to: Classifier_GTDB/Bacteria-gtdb-214-tax.qza


In [25]:
for dom in DOMS:
    f = DOMS[dom][0]
    r = DOMS[dom][1]
    
    !qiime feature-classifier extract-reads \
        --i-sequences $outdir/$dom-gtdb-214-seqs.qza \
        --p-f-primer $f \
        --p-r-primer $r \
        --p-n-jobs 4 \
        --p-read-orientation 'both' \
        --o-reads $outdir/$dom-gtdb-214-seqs.qza

Saved FeatureData[Sequence] to: Classifier_GTDB/Archaea-gtdb-214-seqs.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Bacteria-gtdb-214-seqs.qza


In [28]:
for dom in DOMS:

    !qiime rescript dereplicate \
        --i-sequences $outdir/$dom-gtdb-214-seqs.qza  \
        --i-taxa $outdir/$dom-gtdb-214-tax.qza \
        --p-mode 'uniq' \
        --o-dereplicated-sequences $outdir/$dom-gtdb-214-seqs.qza \
        --o-dereplicated-taxa $outdir/$dom-gtdb-214-tax.qza

Saved FeatureData[Sequence] to: Classifier_GTDB/Archaea-gtdb-214-seqs.qza
Saved FeatureData[Taxonomy] to: Classifier_GTDB/Archaea-gtdb-214-tax.qza
Saved FeatureData[Sequence] to: Classifier_GTDB/Bacteria-gtdb-214-seqs.qza
Saved FeatureData[Taxonomy] to: Classifier_GTDB/Bacteria-gtdb-214-tax.qza


In [29]:
for dom in DOMS:

    !qiime feature-classifier fit-classifier-naive-bayes \
        --i-reference-reads $outdir/$dom-gtdb-214-seqs.qza \
        --i-reference-taxonomy $outdir/$dom-gtdb-214-tax.qza \
        --o-classifier $outdir/$dom-gtdb-214-classifier.qza

Saved TaxonomicClassifier to: Classifier_GTDB/Archaea-gtdb-214-classifier.qza
Saved TaxonomicClassifier to: Classifier_GTDB/Bacteria-gtdb-214-classifier.qza
